### NFI Cumulative Distribution Function plots and statistics for the Glacier Bay VQOR
These for GLBARENDU2011, GLBAMCLEOD2011 (re-analyzed from original [Lynch E. 2012. *Glacier Bay National Park and Preserve: Acoustical Monitoring 2011*](https://irma.nps.gov/DataStore/Reference/Profile/2268564)), and subsequent monitoring GLBARENDU2021, GLBAMCLEOD2021. <br>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import os

# general NSNSD acoustical tools
import sys
sys.path.append(r"C:\Users\DBetchkal\PythonScripts\3_GITHUB_REPOSITORIES\iyore")
sys.path.append(r"C:\Users\DBetchkal\PythonScripts\3_GITHUB_REPOSITORIES\soundDB")
sys.path.append(r"C:\Users\DBetchkal\PythonScripts\3_GITHUB_REPOSITORIES\derivedDataFunctions")
sys.path.append(r"C:\Users\DBetchkal\PythonScripts\3_GITHUB_REPOSITORIES\GLBA-vesselAcoustics")
import iyore
from soundDB import *
from derivedDataFunctions import *
from merge_SRCID import *
from AIS_utilities import contiguous_regions


def VOQR_cumulative_histogram_nfi(ds, unit, site, year):
    
    '''
    Analyze spectrogram annotations for a site to make a CDF plot.
    Work under conditions of the GLBA Vessel Operating and Quota Requirements plan.
    '''

    src = srcid(ds, unit=unit, site=site, year=year).combine()
    src_m = merge_SRCID(src) # merging is important because vessels often span multiple hours

    # extract only vessel events; a condition of the VQoR
    # in this analysis: 
    #     AIS-derived vessels use a 4-character srcID mapped to MMSI (e.g., '0.702')
    #     other vessels have the conventional srcID (i.e., '3.0')
    vessel_events = src_m.loc[(src_m.srcID == 3)|(src_m.srcID.astype('str').str.len() == 5), :].copy()

    # they must be sorted
    vessel_events.sort_index(inplace=True)
    

    # this is by far more straightforward 
    NFIlst = vessel_events.index.to_series().diff()

    # we only want non-negative intervals
    out = pd.Series(np.array([m for m in NFIlst[NFIlst > "00:00:00"]]))

    
    # compute noise-free intervals between all events
    nfis = NFI_list(vessel_events, unit="hours")

    # for comparing sites a constant number of bins is required; let's use 100
    val, base = np.histogram(nfis, bins=np.logspace(-2,2, 200))

    #evaluate the cumulative
    cumu = np.cumsum(val)/max(np.cumsum(val))
    
    # compute time audible as a percentage of the overall record
    # note: this computation assumes partial days have been separated at the archive level
    TA_p = 100*src_m.loc[:, "len"].sum().total_seconds()/(3600*len([e for e in ds.nvspl(unit=u, site=s, year=y)]))
    
    print("80th = {0:.1f} hours".format(np.percentile(nfis, 80)),
          "70th = {0:.1f} hours".format(np.percentile(nfis, 70)),
          "mean = {0:.1f} hours".format(np.mean(nfis)),
          "50th = {0:.1f} hours".format(np.percentile(nfis, 50)),
          "\n\n")
    
    return base, cumu, TA_p, metrics

def VOQR_cumulative_histogram_sel(ds, unit, site, year):
    
    '''
    Analyze spectrogram annotations for a site to make a CDF plot.
    Work under conditions of the GLBA Vessel Operating and Quota Requirements plan.
    '''

    src = srcid(ds, unit=unit, site=site, year=year).combine()
    src = src[~src.index.duplicated(keep='first')]
    src_m = merge_SRCID(src) # merging is important because vessels often span multiple hours

    # extract only vessel events; a condition of the VQoR
    # in this analysis: 
    #     AIS-derived vessels use a 4-character srcID mapped to MMSI (e.g., '0.702')
    #     other vessels have the conventional srcID (i.e., '3.0')
    vessel_events = src_m.loc[(src_m.srcID == 3)|(src_m.srcID.astype('str').str.len() == 5), :].copy()

    # they must be sorted
    vessel_events.sort_index(inplace=True)
    
    sels = [float(val) for val in vessel_events['SELt'].values]
    
    # for comparing sites a constant number of bins is required; let's use 100
    val, base = np.histogram(sels, bins=np.linspace(20, 130, 200))

    #evaluate the cumulative
    cumu = np.cumsum(val)/max(np.cumsum(val))
    
    return base, cumu

### Step One: plot overall vessel NFIs, SELs as Cumulative Distribution Functions

In [ ]:
ds = iyore.Dataset(r"E:")

wdir = r"T:\ResMgmt\WAGS\Sound\Experiments & Data\2022 - 2021 GLBA VQOR Vessel Quotas and Operating Requirements plan"

# =========== plot ===================

nfi_refs = [0.1, 0.2, 0.5, 1, 2, 5, 10, 20, 50, 100] # in hours

fig, ax = plt.subplots(1, 1, figsize=(7, 3.5))

#                   ('GLBA', 'BARTC', 2020, 'mediumblue'), # where are these?
#                   ('GLBA', 'HUTCH', 2020, 'pink'),
# for u, s, y, c in [('GLBA', 'BARTC', 2021, 'blue'),
#                    ('GLBA', 'MCLEOD', 2011, 'mediumvioletred'),
#                    ('GLBA', 'MCLEOD', 2021, 'violet'),
#                    ('GLBA', 'RENDU', 2011, 'mediumspringgreen'),
#                    ('GLBA', 'RENDU', 2020, 'orange'),
#                    ('GLBA', 'RENDU', 2021, 'green'),
#                    ('GLBA', 'REID', 2020, 'indigo'),
#                    ('GLBA', 'BEARD', 2021, 'saddlebrown'),
#                    ('GLBA', 'HUTCH', 2011, 'orangered'),
#                    ('GLBA', 'MOUSE', 2011, 'dodgerblue')]:
    
for u, s, y, c in [('GLBA', 'RENDU', 2011, 'mediumspringgreen'),
                   ('GLBA', 'RENDU', 2021, 'green'),
                   ('GLBA', 'MCLEOD', 2011, 'mediumvioletred'),
                   ('GLBA', 'MCLEOD', 2021, 'violet')]:
    
    print("now processing", u+s+str(y))
    
    # to visualize the annotations we: parse + filter + mine + represent...
    base, cumu, TA_p, metrics = VOQR_cumulative_histogram_nfi(ds, u, s, y)

    # plot the results
    ax.step(base[:-1], cumu, label=u+s+str(y), 
            lw=0.75, color=c)
    

ax.set_xscale('log')
ax.set_xlabel("Noise Free Interval length, hours \n[logarithmic bins]", 
              labelpad=25, fontsize=11)
ax.set_xticks(ticks=nfi_refs)
ax.set_xticklabels(labels=nfi_refs)
ax.set_xlim([0.1, 100])

# add visual cues for familiar lengths of time
# (this helps interpret the logarithmic bins)
for t in nfi_refs: 
    ax.axvline(t, lw=0.5, alpha=0.1, color="k", zorder=-2)

ax.set_ylabel("Percentile", labelpad=25, fontsize=11)
ax.set_ylim([0, 1.01])

ax.axhline(1, lw=0.5, ls="dotted", alpha=0.1, color="k", zorder=-2)

ax.axhline(0.8, lw=0.5, ls="dotted", alpha=0.6, color="k", zorder=-2)
ax.axhline(0.7, lw=0.5, ls="dotted", alpha=0.6, color="k", zorder=-2)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

ax.legend(bbox_to_anchor=(1.1, 0.9818),
          frameon=False)

ax.set_title("Noise Free Interval Between Vessel Events",
             loc="center", y=1.12, fontsize=13)

plt.savefig(wdir+os.sep+"GLBA_VOQR_VesselNFIs_MCLEOD_20221104.png", dpi=200,
            bbox_inches="tight", facecolor="white")
plt.show()

#### Same graphic as above, but for Sound Exposure Level (SEL)
The analysis is identical, but visualizing the results requires different contextualization

In [ ]:
ds = iyore.Dataset(r"E:")

wdir = r"T:\ResMgmt\WAGS\Sound\Experiments & Data\2022 - 2021 GLBA VQOR Vessel Quotas and Operating Requirements plan"

# =========== plot ===================

nfi_refs = [0.1, 0.2, 0.5, 1, 2, 5, 10, 20, 50, 100] # in hours

fig, ax = plt.subplots(1, 1, figsize=(7, 3.5))

for u, s, y, c in [("GLBA", "RENDU", 2011, 'mediumspringgreen'),
                   ("GLBA", "RENDU", 2021, 'green'),
                   ("GLBA", "MCLEOD", 2011, 'mediumvioletred'),
                   ("GLBA", "MCLEOD", 2021, 'violet'),
                   ("GLBA", "BARTC", 2021, 'dodgerblue'),
                   ("GLBA", "BEARD", 2021, 'chocolate')]:
    
    print("now processing", u+s+str(y))
    
    # to visualize the annotations we: parse + filter + mine + represent...
    base, cumu,  = VOQR_cumulative_histogram_sel(ds, u, s, y)
    
    # plot the results
    ax.step(base[:-1], cumu, label=u+s+str(y), 
            lw=0.75, color=c)

# ax.set_xscale('log')
ax.set_xlabel("Sound Exposure Level [12.5 - 1250 Hz] (dB)", 
              labelpad=25, fontsize=11)
# ax.set_xticks(ticks=nfi_refs)
# ax.set_xticklabels(labels=nfi_refs)
ax.set_xlim([30, 100])

# # add visual cues for familiar lengths of time
# # (this helps interpret the logarithmic bins)
# for t in nfi_refs: 
#     ax.axvline(t, lw=0.5, alpha=0.1, color="k", zorder=-2)

ax.set_ylabel("Percentile", labelpad=25, fontsize=11)
ax.set_ylim([0, 1.01])

# ax.axhline(1, lw=0.5, ls="dotted", alpha=0.1, color="k", zorder=-2)
# ax.axhline(0.8, lw=0.5, ls="dotted", alpha=0.6, color="k", zorder=-2)
# ax.axhline(0.7, lw=0.5, ls="dotted", alpha=0.6, color="k", zorder=-2)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

ax.legend(bbox_to_anchor=(1.1, 0.9818),
          frameon=False)

ax.set_title("Sound Exposure Level (=Noise Dose) of Vessel Events",
             loc="center", y=1.12, fontsize=13)

plt.savefig(wdir+os.sep+"GLBA_VOQR_VesselSELsfirstDraft.png", dpi=200,
            bbox_inches="tight", facecolor="white")
plt.show()

### Step Two: Load and filter the annotation record

In [ ]:
ds = iyore.Dataset(r"E:")
u, s, y = "GLBA", "RENDU", 2021

src = srcid(ds, unit=u, site=s, year=y).combine()
src_m = merge_SRCID(src) # merging is important because vessels often span multiple hours

# extract only vessel events; a condition of the VQoR
# in this analysis: 
#     AIS-derived vessels use a 4-character srcID mapped to MMSI (e.g., '0.702')
#     other vessels have the conventional srcID (i.e., '3.0')
vessel_events = src_m.loc[(src_m.srcID == 3)|(src_m.srcID.astype('str').str.len() == 5), :].copy()

# they must be sorted
vessel_events.sort_index(inplace=True)

### Step Three: Load NVSPL record, add a binary series for vessel noise presence

In [ ]:
# load (part of) the acoustic record using `soundDB`
nv = nvspl(ds, unit=u, site=s, year=y).combine()
nv.index = nv.index.droplevel()

# select only the salient data we'll be working with
nv_select = nv.loc[:, "12.5":"dbA"].merge(nv["WindSpeed"], how="left", left_index=True, right_index=True)

# we want to expand all the events into a complete list of their inclusive datetimes
noisy_times = np.array([])
for start, row in vessel_events.iterrows():
    
    event_times = [start + dt.timedelta(seconds=i) for i in np.arange(0, row["len"].total_seconds())]
    noisy_times = np.append(noisy_times, event_times)


# flatten and take only the unique values, as this is a composite effort
noise_series_raw = noisy_times.flatten()
noise_series = np.unique(noise_series_raw)

# compile into a `pandas.Series` object for merging with NVSPL
noise_presence = pd.Series(np.ones(noise_series.shape), index=noise_series, name="presence")

# merge the indicator series with the acoustic data, 
# keeping all rows from the latter and assigning np.nan for all times without noise
indicator_nv = nv_select.merge(noise_presence, how="left", left_index=True, right_index=True)
indicator_nv = indicator_nv.fillna(0) # then convert np.nan -> 0

indicator_nv

In [ ]:
nfis_select=contiguous_regions(indicator_nv['presence'] == 0)
I_max = (nfis_select.T[1] - nfis_select.T[0]).max()/3600

### Step Four: Compute longest diurnal NFI
- group NVSPL record by date, then <br>
- subselect by desired diural period, then <br>
- group `indicator_nv['presence']` by contiguous regions with value `== 0`, that is, *noise-free seconds*
- thus: *length of group* $\equiv$ *length of noise-free interval*
- glean `max(len(group))`

In [ ]:
start_bound = 5 # "05:00"
end_bound = 21 # "21:00" # must be in the same day!

I_maxs = []
nvspl_dates = np.unique(indicator_nv.index.date)
for date in nvspl_dates:
    
    s_time = dt.time(hour=start_bound)
    s_bound = dt.datetime.combine(date, s_time)
    e_time = dt.time(hour=end_bound)
    e_bound = dt.datetime.combine(date, e_time)
    
    print("on", date, "between", s_time, "and", e_time, ":\n")
    print("\tthe longest noise-free interval")
    
    nv_select = indicator_nv.loc[s_bound:e_bound, :]

    try:
        nfis_select=contiguous_regions(nv_select['presence'] == 0)
        I_max = (nfis_select.T[1] - nfis_select.T[0]).max()/3600
        print("\tI_max = {0:.01f} hours".format(I_max))
        print("\n\n")
        I_maxs.append(I_max)
        
    except IndexError:
        print("no noise-free periods during the timeframe indicated! \n\n")
        I_maxs.append(np.nan)

longest = pd.DataFrame(np.array([I_maxs]).T, 
                       index=nvspl_dates, columns=["I_max(hrs)"])
longest

### Step Five: plot longest-NFI results for a deployment
Scale to length of diurnal period

In [ ]:
period_len = end_bound-start_bound

plt.figure(figsize=(12, 2.5))
plt.plot(longest.index, longest["I_max(hrs)"], marker="o", ms=3, color="k", ls="")
plt.axhline(period_len, ls="--", color="gray", alpha=0.2, zorder=-1)
plt.text(longest.index.min(), period_len+0.8, 
         "maximum achievable length = {0:.1f} hrs".format(period_len), 
         color="gray", alpha=0.4, zorder=-1)
plt.ylim([0, period_len + 4])
plt.ylabel("Maximum Noise-Free\nInterval Observed (hrs)", labelpad=15)
plt.title(u+s+str(y)+" maximum diurnal NFI", loc="left", y=1.04)
plt.savefig(r"C:\Users\DBetchkal\Desktop\MaximumDiurnalNFI_"+u+s+str(y)+".png",
            dpi=150, bbox_inches="tight", facecolor="white")
plt.show()

# format tabular report
print("n days observed: {0} days".format(len(longest)), "\n",
      "min observed: {0:.1f} hours".format(longest["I_max(hrs)"].min()), "\n",
      "20th percentile: {0:.1f} hours".format(longest["I_max(hrs)"].quantile(0.2)), "\n",
      "median: {0:.1f} hours".format(longest["I_max(hrs)"].quantile(0.5)), "\n",
      "mean: {0:.1f} hours".format(longest["I_max(hrs)"].mean()), "\n",
      "80th percentile: {0:.1f} hours".format(longest["I_max(hrs)"].quantile(0.8)), "\n",
      "maximum achievable: {0:.1f} hours".format(period_len), "\n", 
      "was achieved?", (longest["I_max(hrs)"] >= period_len).sum() > 0)